In [0]:
!pip3 uninstall keras
!pip3 install keras==2.2.4


# Atividade Named-Entity Recognition (NER)

### Desafio

Crie a rotina para processamento dos textos, preparando os documentos para respeitar a estrutura proposta para rodar uma rede neural capaz de realizar a atividade de reconhecimento de entidades.

Lembre que o resultado deverá ter três colunas (identificador, palavra e tag) como exemplificado na tabela abaixo:

| identificador | palavra | tag |
|:--:|:--:|:--:|
|102|tronco|O|
|102|da|O|
|102|coronaria|vaso|
|102|esquerda|vaso|
|102|com|O|
|102|trajeto|O|

Você deverá considerar as seguintes classes e termos por classe:

-   **vaso**:
	- descendente anterior;
	- coronaria direita;
	- coronaria esquerda;
	- circunflexa;
	- primeiro ramo marginal;
	- segundo ramo marginal;
	- terceiro ramo marginal;
	- primeiro ramo diagonal;
	- segundo ramo diagonal;
	- terceiro ramo diagonal;
	- ventricular posterior;
	- arteria diagonalis;
	- descendente posteiror.
-   **trajeto**:
	- trajeto intramiocardico;
	- origem;
	- retroaortico;
	- interaortopulmonar;
	- valsalva;
	- seio;
	- sinotubular.
-   **placa**:
	- placa;
	- placas;
	- ateromatose.
-   **composição**:
	- calcificada;
	- calcificadas;
	- densamente calcificada;
	- densamente calcificadas;
	- densa;
	- densamente calcificado;
	- parcialmente calcificada;
	- predominantemente calcificada;
	- predominantemente calcificadas;
	- mista;
	- mistas;
	- predominantemente nao calcificada;
	- predominantemente não calcificada;
	- parcialmente não calcificada;
	- principalmente calcificada;
	- principalmente não calcificada;
	- principalmente nao calcificada;
	- predominio calcificado;
	- predominio nao calcificado;
	- não calcificada.
-   **grau**:
	- sem redução luminal;
	- discreta;
	- irregularidades parietais;
	- irregularidade parietal;
	- menor que 50%;
	- irregularidades luminais;
	- irregularidade luminal;
	- irregularidade;
	- irregularidades;
	- proxima de 50%;
	- proximo de 50%;
	- entre 50 e 70%;
	- 50%;
	- acima de 50%;
	- maior que 50%;
	- ao redor de 50%;
	- cerca de 50%;
	- em torno de 50%;
	- acima de 70%;
	- 70%;
	- cerca de 70%;
	- ao redor de 70%;
	- em torno de 70%;
	- moderada;
	- moderada reducao luminal;
	- reducao luminal moderada;
	- estenose moderada;
	- grau pelo menos moderado;
	- grau moderado;
	- acentuada;
	- suboclusao;
	- reducao luminal critica;
	- estenose critica;
	- oclusao;
	- ocluido;
	- ocluida.
-   **modificador V**:
	- modificador V;
	- vulnerabilidade;
	- remodelamento positivo;
	- baixa atenuação;
	- napking ring;
	- anel de guardanapo;
	- spot calcifications;
	- remodelamento arterial positivo.
-   **stent**:
	- stent.
-   **redução stent**:
	- hiperplasia neointimal;
	- neointimal;
	- proliferação neointimal.
-   **enxerto**:
	- enxerto.

In [0]:
# Resolução
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_rows', None)

df = pd.read_excel('https://github.com/pgiaeinstein/ner-aula/raw/master/data_coronaria.xlsx')

In [6]:
df.shape

(439, 1)

In [8]:
df.head()

,texto
0,paciente com escore de calcio de zero fase com...
1,paciente com escore de calcio de zero fase com...
2,paciente com escore de calcio de zero fase com...
3,paciente com escore de calcio zero fase com co...
4,paciente com escore de calcio de zero fase com...


In [0]:
reg_vaso = re.compile(r"(descendente anterior|descendente posterior|coronaria (esquerda|direita)|circunflexa|ventricular posterior|arteria diagonalis|((primeiro|segundo|terceiro|quarto)(\se\s|\s))+(ramos?)?\s?(diagon(al|ais)?|margin(al|ais)?)?)")
reg_trajeto = re.compile(r"(trajeto intramiocardico|retroaórtico|interaortopulmonar|valssalva|sinotubular|seio)")
reg_placa = re.compile(r"(placas?|ateromatose)")
reg_desc_placa = re.compile(r"((predominantemente nao |nao |parcialmente |parcialmente nao |predominantemente |principalmente |principalmente nao |densamente )?calcificadas?|mistas?|predominio calcificado|predominio nao calcificado|densamente calcificado|densa)")
reg_oclusao = re.compile(r"(oclusao|suboclusao|acentuada|\b70%?\b|\b50%?\b|moderada\s?(reducao)?\s?(luminal)?|discreta\b|(irregularidades?\s?(pariet(ais|al)|lumin(al|ais))?)|menor que 50%?|maior que 50%?|entre 50%? e 70%?|acima de 50%?|ao redor de 50%?|cerca de 50%?|em torno de 50%?|acima de 70%?|cerca de 70%?|ao redor de 70%?|em torno de 70%?|reducao luminal critica|estenose critica|ocluid(o|a)|reducao luminal moderada|estenose moderada|grau pelo menos moderado|grau moderado|proxima de 50%?|proximo de 50%?)")
reg_modificador = re.compile(r"(modificador v|vulnerabilidade|remodelamento positivo|baixa atenuacao|napking ring|anel de guardanapo|spot calcifications|baixa atenuacao|remodelamento arterial positivo)")
reg_stent = re.compile(r"(stent)")
reg_red_stent = re.compile(r"(proliferacao neointimal|hiperplasia neointimal|hiperplasia|neointimal)")
reg_enxerto = re.compile(r"(enxerto)")

In [0]:
lista_documentos = df['texto'].tolist()

In [0]:
def encontra_tags(texto):
    
    lista_retorno = list()
    
    for item in reg_vaso.finditer(texto):
        lista_retorno.append({
            'classe'  : 'vaso',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
    
    for item in reg_trajeto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'trajeto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_desc_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'desc_placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_oclusao.finditer(texto):
        lista_retorno.append({
            'classe'  : 'oclusao',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_modificador.finditer(texto):
        lista_retorno.append({
            'classe'  : 'modificador',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_red_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'red_stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_enxerto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'enxerto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })

    return sorted(lista_retorno, key = lambda x : x['inicial'])

In [0]:
def parse_texto(texto, lista_tags, indice):
    
    lista_retorno = list()
    lista_tmp = list()
    
    tamanho_texto = len(texto)
    tamanho_lista = len(lista_tags)
    
    ultima_pos = 0
    
    for index, achado in enumerate(lista_tags):
        
        if achado['inicial'] != 0 and index == 0:
            lista_tmp.append((texto[0:achado['inicial']], 'O'))
            ultima_pos = achado['inicial']

        if ultima_pos != achado['inicial']:
            lista_tmp.append((texto[ultima_pos:achado['inicial']], 'O'))
            
        lista_tmp.append((texto[achado['inicial']:achado['final']], achado['classe']))
        ultima_pos = achado['final']
            
        if index + 1 == tamanho_lista:
            lista_tmp.append((texto[ultima_pos:], 'O'))
        
    for item in lista_tmp:
        tag_atual = item[1]
        for palavra in item[0].split(' '):
            if palavra == '':
                continue
            lista_retorno.append({
                'identificador' : indice,
                'palavra'       : palavra,
                'tag'           : tag_atual
            }) 
    
    return lista_retorno
        

In [0]:
index = 1
lista_resultante = list()

for documento in lista_documentos:
    lista_tags_documento = encontra_tags(documento)
    lista_resultante.extend(parse_texto(documento, lista_tags_documento, index))
    index += 1

In [0]:
df_para_nn = pd.DataFrame(lista_resultante)

In [15]:
df_para_nn.head(200)

,identificador,palavra,tag
0,1,paciente,O
1,1,com,O
2,1,escore,O
3,1,de,O
4,1,calcio,O
5,1,de,O
6,1,zero,O
7,1,fase,O
8,1,com,O
9,1,contraste,O


### Modelos

In [0]:
import pandas as pd
import numpy as np

# data = pd.read_excel("new_output.xlsx")

In [0]:
data = df_para_nn

In [0]:
# data = data.fillna(method="ffill")

In [18]:
data.tail(10)

,identificador,palavra,tag
48523,439,do,O
48524,439,mesmo,O
48525,439,lado,O
48526,439,compativel,O
48527,439,com,O
48528,439,sequela,O
48529,439,de,O
48530,439,complexo,O
48531,439,granulomatoso,O
48532,439,primario,O


In [15]:
f'Total de documentos na base: {len(data.identificador.unique())}'

'Total de documentos na base: 439'

In [19]:
words = list(set(data["palavra"].values))
words.append("ENDPAD")
n_words = len(words)
n_words

638

In [20]:
tags = list(set(data["tag"].values))
n_tags = len(tags)
n_tags

7

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["palavra"].values.tolist(),
                                                     s["tag"].values.tolist())]
        self.grouped = self.data.groupby("identificador").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["identificador".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data)

In [0]:
sent = getter.get_next()

In [0]:
sentences = getter.sentences

In [0]:
max_len = 300
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [0]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [28]:
word2idx['ENDPAD']

637

In [29]:
len(X[0])

300

In [0]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [0]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [32]:
lista_documentos[1]

'paciente com escore de calcio de zero fase com contraste coronaria direita dominante tronco da coronaria esquerda com trajeto e calibre normais bifurcando em descendente anterior e circunflexa arteria descendente anterior com trajeto e calibre normais primeiro e segundo ramos diagonais de pequeno calibre ambos com discretas irregularidades parietais proximais arteria circunflexa com trajeto e calibre normais primeiro ramo marginal de pequeno calibre sem reducao luminal segundo ramo marginal de grande calibre sem reducao luminal coronaria direita com trajeto e calibre normais arterias descendente posterior e ventricular posterior com trajeto e calibre normais'

In [33]:
y[1]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 5, 5, 5, 2, 2, 5, 5, 5, 5, 5,
       5, 5, 2, 2, 5, 2, 5, 2, 2, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 5, 5, 5,
       5, 5, 5, 0, 0, 5, 5, 2, 5, 5, 5, 5, 5, 2, 2, 2, 5, 5, 5, 5, 5, 5,
       2, 2, 2, 5, 5, 5, 5, 5, 5, 2, 2, 5, 5, 5, 5, 5, 5, 2, 2, 5, 2, 2,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], d

In [34]:
tag2idx

{'O': 5,
 'desc_placa': 1,
 'modificador': 6,
 'oclusao': 0,
 'placa': 3,
 'trajeto': 4,
 'vaso': 2}

In [0]:
from tensorflow.keras.utils import to_categorical

In [0]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=6)

### BI LSTM

In [0]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [37]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=32, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model = Model(input, out)

In [0]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [40]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=30, validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 355 samples, validate on 40 samples
Epoch 1/15
355/355 [==============================] - 15s 43ms/sample - loss: 1.0170 - acc: 0.8248 - val_loss: 0.3672 - val_acc: 0.8999
Epoch 2/15
355/355 [==============================] - 12s 34ms/sample - loss: 0.3390 - acc: 0.8955 - val_loss: 0.2868 - val_acc: 0.8999
Epoch 3/15
355/355 [==============================] - 13s 35ms/sample - loss: 0.3011 - acc: 0.8955 - val_loss: 0.2764 - val_acc: 0.8999
Epoch 4/15
355/355 [==============================] - 13s 36ms/sample - loss: 0.2884 - acc: 0.8955 - val_loss: 0.2697 - val_acc: 0.8999
Epoch 5/15
355/355 [==============================] - 13s 37ms/sample - loss: 0.2755 - acc: 0.8955 - val_loss: 0.2482 - val_acc: 0.8999
Epoch 6/15
355/355 [==============================] - 13s 36ms/sample - loss: 0.2575 - acc: 0.8964 - val_loss: 0.2248 - val_acc: 0.9009
Epoch 7/15
355/355 [=========================

In [42]:
!pip3 install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=e9963712e1ad4d893ad1def74fe505c2df513d76a6a5e6f1f87812b804870090
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [44]:
test_pred = model.predict(X_te, verbose=1)

44/44 [==============================] - 1s 15ms/sample


In [40]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

NameError: ignored

In [46]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

ACC      : 98.6288%
PRECISION: 80.6748%
RECALL   : 78.2738%
F1       : 79.4562%


In [47]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

       vaso       0.85      0.95      0.89       534
      placa       1.00      0.03      0.06        34
 desc_placa       0.08      0.06      0.07        34
    oclusao       0.61      0.35      0.44        49
    trajeto       0.00      0.00      0.00        20
modificador       0.00      0.00      0.00         1

  micro avg       0.81      0.78      0.79       672
  macro avg       0.77      0.78      0.75       672



### BI-LSTM-CRF

In [62]:
!git clone https://www.github.com/keras-team/keras-contrib.git && cd keras-contrib && python3 setup.py install

Cloning into 'keras-contrib'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 3634 (delta 6), reused 0 (delta 0), pack-reused 3622
Receiving objects: 100% (3634/3634), 862.51 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (2328/2328), done.
running install
running bdist_egg
running egg_info
creating keras_contrib.egg-info
writing keras_contrib.egg-info/PKG-INFO
writing dependency_links to keras_contrib.egg-info/dependency_links.txt
writing requirements to keras_contrib.egg-info/requires.txt
writing top-level names to keras_contrib.egg-info/top_level.txt
writing manifest file 'keras_contrib.egg-info/SOURCES.txt'
writing manifest file 'keras_contrib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/keras_contrib
copying keras_contrib/__init__.py -> build/lib/keras_contrib

In [0]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [42]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=32, input_length=max_len, mask_zero=True)(input)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
# model = TimeDistributed(Dense(64, activation="relu"))(model)
crf = CRF(n_tags)
out = crf(model)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model = Model(input, out)

In [44]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/usr/local/lib/python3.6/dist-packages/keras_contrib-2.0.8-py3.6.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
/usr/local/lib/python3.6/dist-packages/keras_contrib-2.0.8-py3.6.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 32)           20448     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 128)          49664     
_________________________________________________________________
crf_1 (CRF)                  (None, 300, 7)            966       
Total params: 71,078
Trainable params: 71,078
Non-trainable params: 0
_________________________________________________________________


In [46]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=30, validation_split=0.1, verbose=1)




Train on 355 samples, validate on 40 samples
Epoch 1/15





355/355 [==============================] - 27s 75ms/step - loss: 1.2448 - crf_viterbi_accuracy: 0.5193 - val_loss: 0.3746 - val_crf_viterbi_accuracy: 0.8999
Epoch 2/15
355/355 [==============================] - 23s 65ms/step - loss: 0.3689 - crf_viterbi_accuracy: 0.8955 - val_loss: 0.3223 - val_crf_viterbi_accuracy: 0.8999
Epoch 3/15
355/355 [==============================] - 24s 66ms/step - loss: 0.3368 - crf_viterbi_accuracy: 0.8955 - val_loss: 0.3091 - val_crf_viterbi_accuracy: 0.8999
Epoch 4/15
355/355 [==============================] - 24s 66ms/step - loss: 0.3214 - crf_viterbi_accuracy: 0.8955 - val_loss: 0.2983 - val_crf_viterbi_accuracy: 0.9004
Epoch 5/15
355/355 [==============================] - 23s 66ms/step - loss: 0.3154 - crf_viterbi_accuracy: 0.8962 - val_loss: 0.2878 - val_crf_viterbi_accuracy: 0.8999
Epoch 6/15
355/355 [==============================] - 23s 64ms/step - loss: 0.3007 - crf_viterbi_accuracy: 

In [0]:
hist = pd.DataFrame(history.history)

In [0]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [49]:
test_pred = model.predict(X_te, verbose=1)

44/44 [==============================] - 2s 40ms/step


In [0]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [51]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

ACC      : 98.3864%
PRECISION: 76.6272%
RECALL   : 77.0833%
F1       : 76.8546%


In [52]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

       vaso       0.80      0.95      0.87       534
      placa       0.00      0.00      0.00        34
    oclusao       0.39      0.22      0.29        49
    trajeto       0.00      0.00      0.00        20
 desc_placa       0.10      0.06      0.07        34
modificador       0.00      0.00      0.00         1

  micro avg       0.77      0.77      0.77       672
  macro avg       0.67      0.77      0.72       672

